In [1]:
import pandas as pd
import json
from pathlib import Path
import requests
import os

In [32]:
# Load the provinces data from the tat_thai_provinces.json file
with open('../../dataset/tat_v1_provinces.json', 'r', encoding='utf-8') as f:
    provinces = json.load(f)

# API key for authentication
API_KEY = 'UmYQ8yiaTyArXEN5GMRJiCcvtu99ILDd'

# Base URL for fetching attractions list
BASE_URL = 'https://tatdataapi.io/api/v1/places'

LANGUAGE = 'th'

# Headers for the API request
headers = {
    'accept': 'application/json',
    'Accept-Language': LANGUAGE,
    'x-api-key': API_KEY,
}


In [33]:
# Loop through each province and fetch the data
for province in provinces:
    province_id = province['provinceId']
    province_name = province['name']
    
    # Prepare the parameters for the API request
    params = {
        'provinceId': province_id,
        # 'place_category_id': 3,  # You can change this if necessary
        'pageSize': 100, # For V1
        # 'page': 1,
        'pageNumber': 1, # For V1
        # 'sort_by': 'placeId',
        # 'categoryCode': 'attractions' # For V2
        'categoryCode': 'attraction' # For V1
    }
    
    # Send the GET request to fetch the attractions
    response = requests.get(BASE_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        
        # Check if 'data' is available in the response
        if 'data' in data:
            attractions = data['data']
            
            # Transform the data into a DataFrame
            records = []
            for item in attractions:
                location = item.get("location", {})
                tags = item.get("tags", [])
                category = item.get("category", {})
                province = location.get("province", {})
                district = location.get("district", {})
                sub_district = location.get("subDistrict", {})
                
                record = {
                    "placeId": item.get("placeId"),
                    "place_name": item.get("name"),
                    "introduction": item.get("introduction"),
                    "category_name": category.get("name"),
                    "categoryId": category.get("categoryId"),
                    "latitude": item.get("latitude"),
                    "longitude": item.get("longitude"),
                    "postcode": location.get("postcode") if location else None,
                    "thumbnail_url": item.get("thumbnailUrl", [None])[0] if item.get("thumbnailUrl") else None,
                    "tags": ",".join(tags) if tags else "",  # Safely handle tags
                    "province_Id": province.get("provinceId") if province else None,
                    "province_name": province.get("name") if province else None,
                    "district_Id": district.get("districtId") if district else None,
                    "district_name": district.get("name") if district else None,
                    "sub_district_Id": sub_district.get("subDistrictId") if sub_district else None,
                    "sub_district_name": sub_district.get("name") if sub_district else None,
                    "updated_at": item.get("updatedAt"),
                }
                records.append(record)
            
            # Create a DataFrame
            df = pd.DataFrame(records)
            
            # Save the DataFrame to a CSV file
            csv_filename = f"data/{province_id}_attractions_{LANGUAGE}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8')
            print(f"Data for {province_name} saved to {csv_filename}")
        else:
            print(f"No data found for {province_name} (ID: {province_id})")
    else:
        print(f"Failed to fetch data for {province_name} (ID: {province_id})")

Data for ตาก saved to data/115_attractions_th.csv
Data for เลย saved to data/576_attractions_th.csv
Data for ตรัง saved to data/355_attractions_th.csv
Data for ตราด saved to data/466_attractions_th.csv
Data for น่าน saved to data/108_attractions_th.csv
Data for ยะลา saved to data/356_attractions_th.csv
Data for สตูล saved to data/352_attractions_th.csv
Data for แพร่ saved to data/113_attractions_th.csv
Data for ชุมพร saved to data/343_attractions_th.csv
Data for พะเยา saved to data/109_attractions_th.csv
Data for พังงา saved to data/348_attractions_th.csv
Data for ยโสธร saved to data/589_attractions_th.csv
Data for ระนอง saved to data/351_attractions_th.csv
Data for ระยอง saved to data/465_attractions_th.csv
Data for ลำปาง saved to data/104_attractions_th.csv
Data for ลำพูน saved to data/105_attractions_th.csv
Data for สงขลา saved to data/353_attractions_th.csv
Data for กระบี่ saved to data/344_attractions_th.csv
Data for ชลบุรี saved to data/464_attractions_th.csv
Data for ชัยนาท save

In [34]:
# Directory where the CSV files are stored (you can set this to the path where your CSV files are)
csv_directory = './data/'  # Assuming the files are in the current working directory

# Initialize an empty list to hold DataFrames
df_list = []

# Loop through all CSV files in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith(f"_attractions_{LANGUAGE}.csv"):  # Only process the relevant CSV files
        file_path = os.path.join(csv_directory, filename)
        
        # Read each CSV file into a DataFrame and append it to the list
        df = pd.read_csv(file_path)
        df_list.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(df_list, ignore_index=True)

# Save the merged DataFrame to a single CSV file
merged_df.to_csv(f"tat_attractions_{LANGUAGE}.csv", index=False, encoding='utf-8')

print(f"All files have been merged into 'tat_attractions_{LANGUAGE}.csv'")

All files have been merged into 'tat_attractions_th.csv'


In [2]:
# Define the path to the JSON file
json_file = Path("../../dataset/tat_attractions.json")

# Check if the file exists
if not json_file.is_file():
    raise FileNotFoundError(f"The file '{json_file}' does not exist. Please provide the correct file path.")

# Load the JSON file
with json_file.open('r', encoding='utf-8') as file:
    data = json.load(file)

In [7]:

records = []
for item in data['data']:  # Added safeguard for 'data' key
    if not item:  # Ensure item is not None or empty
        continue
    
    location = item.get("location", {})  # Safe way to access location, defaults to empty dict if None
    tags = item.get("tags", [])  # Ensure tags defaults to an empty list
    category = item.get("category", {})  # Ensure category defaults to an empty dict
    province = location.get("province", {})  # Safely get province
    district = location.get("district", {})  # Safely get district
    sub_district = location.get("subDistrict", {})  # Safely get subDistrict
    
    record = {
        "placeId": item.get("placeId"),
        "place_name": item.get("name"),
        "introduction": item.get("introduction"),
        "category_name": category.get("name"),  # Safely access category name
        "categoryId": category.get("categoryId"),  # Safely access category ID
        "latitude": item.get("latitude"),
        "longitude": item.get("longitude"),
        "postcode": location.get("postcode") if location else None,
        "thumbnail_url": item.get("thumbnailUrl", [None])[0] if item.get("thumbnailUrl") else None,
        "tags": ",".join(tags) if tags else "",  # Safely handle tags and avoid errors when None
        "province_Id": province.get("provinceId") if province else None,  # Safely get province ID
        "province_name": province.get("name") if province else None,  # Safely get province name
        "district_Id": district.get("districtId") if district else None,  # Safely get district ID
        "district_name": district.get("name") if district else None,  # Safely get district name
        "sub_district_Id": sub_district.get("subDistrictId") if sub_district else None,  # Safely get sub-district ID
        "sub_district_name": sub_district.get("name") if sub_district else None,  # Safely get sub-district name
        "updated_at": item.get("updatedAt"),
    }
    records.append(record)

# Create a DataFrame from the extracted records
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file (optional)
df.to_csv("tat_attractions.csv", index=False, encoding='utf-8')

# Display the DataFrame to verify
print(df.head())

  placeId                             place_name  \
0    1858  วัดสระเกศ ราชวรมหาวิหาร (วัดภูเขาทอง)   
1   33400               ศาลเจ้าพ่อ-เจ้าแม่ชุมแสง   
2   33586                   ตลาดเก่าร้อยปีชุมแสง   
3   33885                            ตลาดเจ้าค่ะ   
4   33325                 อุทยานเทวสถานพระพิฆเนศ   

                                        introduction      category_name  \
0  วัดสระเกศ หรือ วัดภูเขาทอง (Temple of the Gold...  สถานที่ท่องเที่ยว   
1                                               None  สถานที่ท่องเที่ยว   
2                                               None  สถานที่ท่องเที่ยว   
3  ตั้งอยู่ตำบลเขาดิน บริเวณริมแม่น้ำปิงด้านหน้าว...  สถานที่ท่องเที่ยว   
4                                               None  สถานที่ท่องเที่ยว   

   categoryId  latitude  longitude postcode  \
0           3  13.75389  100.50841    10100   
1           3  15.89933  100.30886    60120   
2           3  15.89541  100.30847    60120   
3           3  15.81700  100.07715    60230   
4

In [47]:
EN_Attractions_df = pd.read_csv('./tat_attractions_en.csv')
TH_Attractions_df = pd.read_csv('./tat_attractions_th.csv')

In [48]:
TH_Attractions_df.head(5)

,placeId,place_name,introduction,category_name,categoryId,latitude,longitude,postcode,thumbnail_url,tags,province_Id,province_name,district_Id,district_name,sub_district_Id,sub_district_name,updated_at
0,P03012583,ถ้ำมโหฬาร,อยู่ที่บ้านหนองหิน ตำบลหนองหิน อยู่ในบริเวณวัด...,สถานที่ท่องเที่ยว,3,17.10731,101.87997,42190.0,h,NaN,576,เลย,4214.0,หนองหิน,5102.0,หนองหิน,2024-07-16T20:03:56.009Z
1,P03014048,เสมาหินทราย บ้านปากแบ่งและบ้านนาหลัก,ที่บ้านนาหลัก เป็นเสมาหินปักคู่สองหลัก แบบที่ม...,สถานที่ท่องเที่ยว,3,17.48602,101.72230,42000.0,h,NaN,576,เลย,4201.0,เมืองเลย,4994.0,กุดป่อง,2024-07-16T20:03:55.442Z
2,P03025545,ศูนย์บริการนักท่องเที่ยววังกวาง,เป็นลานกางเต็นท์และบ้านพักของอุทยานฯ ซึ่งตั้งอ...,สถานที่ท่องเที่ยว,3,16.88681,101.78800,42180.0,h,NaN,576,เลย,4210.0,ภูกระดึง,5076.0,ศรีฐาน,2024-07-16T20:03:51.616Z
3,P03012596,อุทยานแห่งชาติภูสวนทราย,เป็นป่าดิบเขาเป็นพรมแดนธรรมชาติระหว่างแผ่นดินไ...,สถานที่ท่องเที่ยว,3,17.50356,100.93892,42170.0,h,อุทยานแห่งชาติภูสวนทราย,576,เลย,4206.0,นาแห้ว,5044.0,แสงภา,2024-07-16T20:03:51.358Z
4,P03001063,วัดท่าแขก วัดโบราณ,เป็นวัดเก่าแก่อยู่ริมฝั่งแม่น้ำโขง ห่างจากอำเภ...,สถานที่ท่องเที่ยว,3,17.90506,101.68336,42110.0,h,NaN,576,เลย,4203.0,เชียงคาน,5012.0,เชียงคาน,2024-07-16T20:03:50.031Z


In [37]:
EN_Attractions_df.head(5)

,placeId,place_name,introduction,category_name,categoryId,latitude,longitude,postcode,thumbnail_url,tags,province_Id,province_name,district_Id,district_name,sub_district_Id,sub_district_name,updated_at
0,P03018449,Old Town by Phetchaburi’s River CBT,NaN,Attraction,3,13.10722,99.94842,76000.0,h,"เพชรบุรี,ย่านเมืองเก่า",228,Phetchaburi,7601.0,Mueang Phetchaburi,8421.0,Khlong Krachaeng,2024-08-01T20:00:06.292Z
1,P03013153,Kwangchow Floating Market,NaN,Attraction,3,13.11103,99.63627,76160.0,h,NaN,228,Phetchaburi,7603.0,Nong Ya Plong,8459.0,Yang Nam Klat Nuea,2024-07-16T20:03:55.831Z
2,P03014487,Wat Khoi,NaN,Attraction,3,13.10870,99.94154,76000.0,h,"พระธาตุฉิมพลีพระเศรษฐีนวโกฏิ ,เพชรบุรี,วัดข่อย",228,Phetchaburi,7601.0,Mueang Phetchaburi,8421.0,Khlong Krachaeng,2024-07-16T20:03:50.833Z
3,P03013393,Hat Chao Samran,"This beach, which is located approximately 15 ...",Attraction,3,13.00624,100.06602,76100.0,h,NaN,228,Phetchaburi,7601.0,Mueang Phetchaburi,8436.0,Hat Chao Samran,2024-07-16T20:03:47.548Z
4,P03008661,Laem Leaw,NaN,Attraction,3,13.22476,99.99306,76110.0,h,NaN,228,Phetchaburi,7607.0,Ban Laem,8510.0,Ban Laem,2024-07-16T20:03:42.554Z


In [49]:
TH_Attractions_df['placeId'] = TH_Attractions_df['placeId'].astype(str)
EN_Attractions_df['placeId'] = EN_Attractions_df['placeId'].astype(str)

TH_Attractions_df.rename(columns={'introduction': 'introduction_th'}, inplace=True)
TH_Attractions_df.rename(columns={'place_name': 'place_name_th'}, inplace=True)
TH_Attractions_df.rename(columns={'province_name': 'province_name_th'}, inplace=True)
TH_Attractions_df.rename(columns={'district_name': 'district_name_th'}, inplace=True)
TH_Attractions_df.rename(columns={'sub_district_name': 'sub_district_name_th'}, inplace=True)
Merge_Attractions_df = TH_Attractions_df.merge(EN_Attractions_df[['placeId', 'introduction', 'place_name', 'province_name', 'district_name', 'sub_district_name']], on='placeId', how='left')
Merge_Attractions_df.rename(columns={'introduction': 'introduction_en'}, inplace=True)
Merge_Attractions_df.rename(columns={'place_name': 'place_name_en'}, inplace=True)
Merge_Attractions_df.rename(columns={'province_name': 'province_name_en'}, inplace=True)
Merge_Attractions_df.rename(columns={'district_name': 'district_name_en'}, inplace=True)
Merge_Attractions_df.rename(columns={'sub_district_name': 'sub_district_name_en'}, inplace=True)

Merge_Attractions_df['introduction_en'].fillna('-')
Merge_Attractions_df['introduction_th'].fillna('-')

0       อยู่ที่บ้านหนองหิน ตำบลหนองหิน อยู่ในบริเวณวัด...
1       ที่บ้านนาหลัก เป็นเสมาหินปักคู่สองหลัก แบบที่ม...
2       เป็นลานกางเต็นท์และบ้านพักของอุทยานฯ ซึ่งตั้งอ...
3       เป็นป่าดิบเขาเป็นพรมแดนธรรมชาติระหว่างแผ่นดินไ...
4       เป็นวัดเก่าแก่อยู่ริมฝั่งแม่น้ำโขง ห่างจากอำเภ...
                              ...                        
4320                                                    -
4321    ประดิษฐาน "พระบาง" พระพุทธรูปสำคัญคู่บ้านคู่เม...
4322                                                    -
4323                                                    -
4324                                                    -
Name: introduction_th, Length: 4325, dtype: object

In [50]:
Merge_Attractions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4325 entries, 0 to 4324
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   placeId               4325 non-null   object 
 1   place_name_th         4324 non-null   object 
 2   introduction_th       2546 non-null   object 
 3   category_name         4325 non-null   object 
 4   categoryId            4325 non-null   int64  
 5   latitude              4313 non-null   float64
 6   longitude             4313 non-null   float64
 7   postcode              4215 non-null   float64
 8   thumbnail_url         3601 non-null   object 
 9   tags                  857 non-null    object 
 10  province_Id           4325 non-null   int64  
 11  province_name_th      4325 non-null   object 
 12  district_Id           4317 non-null   float64
 13  district_name_th      4317 non-null   object 
 14  sub_district_Id       4217 non-null   float64
 15  sub_district_name_th 

In [51]:
Merge_Attractions_df

,placeId,place_name_th,introduction_th,category_name,categoryId,latitude,longitude,postcode,thumbnail_url,tags,...,district_Id,district_name_th,sub_district_Id,sub_district_name_th,updated_at,introduction_en,place_name_en,province_name_en,district_name_en,sub_district_name_en
0,P03012583,ถ้ำมโหฬาร,อยู่ที่บ้านหนองหิน ตำบลหนองหิน อยู่ในบริเวณวัด...,สถานที่ท่องเที่ยว,3,17.10731,101.879970,42190.0,h,NaN,...,4214.0,หนองหิน,5102.0,หนองหิน,2024-07-16T20:03:56.009Z,"It is at Baan Nong Hin, Tambon Nong Hin, in Ma...",Tham Mahoran,Loei,Nong Hin,Nong Hin
1,P03014048,เสมาหินทราย บ้านปากแบ่งและบ้านนาหลัก,ที่บ้านนาหลัก เป็นเสมาหินปักคู่สองหลัก แบบที่ม...,สถานที่ท่องเที่ยว,3,17.48602,101.722300,42000.0,h,NaN,...,4201.0,เมืองเลย,4994.0,กุดป่อง,2024-07-16T20:03:55.442Z,NaN,Sandstone Semas of Ban Pak Baeng and Ban Na Lak,Loei,Mueang Loei,Kut Pong
2,P03025545,ศูนย์บริการนักท่องเที่ยววังกวาง,เป็นลานกางเต็นท์และบ้านพักของอุทยานฯ ซึ่งตั้งอ...,สถานที่ท่องเที่ยว,3,16.88681,101.788000,42180.0,h,NaN,...,4210.0,ภูกระดึง,5076.0,ศรีฐาน,2024-07-16T20:03:51.616Z,Accommodation and camping right near the peak ...,Wang Kwang Tourist Information Service Center,Loei,Phu Kradueng,Si Than
3,P03012596,อุทยานแห่งชาติภูสวนทราย,เป็นป่าดิบเขาเป็นพรมแดนธรรมชาติระหว่างแผ่นดินไ...,สถานที่ท่องเที่ยว,3,17.50356,100.938920,42170.0,h,อุทยานแห่งชาติภูสวนทราย,...,4206.0,นาแห้ว,5044.0,แสงภา,2024-07-16T20:03:51.358Z,NaN,Phu Suan Sai National Park,Loei,Na Haeo,Saeng Pha
4,P03001063,วัดท่าแขก วัดโบราณ,เป็นวัดเก่าแก่อยู่ริมฝั่งแม่น้ำโขง ห่างจากอำเภ...,สถานที่ท่องเที่ยว,3,17.90506,101.683360,42110.0,h,NaN,...,4203.0,เชียงคาน,5012.0,เชียงคาน,2024-07-16T20:03:50.031Z,NaN,"Wat Tha Khaek, The Ancient Temple",Loei,Chiang Khan,Chiang Khan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,P03025745,แหล่งเรียนรู้โฮงคำสุวรรณเชษฐา,NaN,สถานที่ท่องเที่ยว,3,17.56720,102.285740,39270.0,NaN,NaN,...,3905.0,สุวรรณคูหา,4483.0,สุวรรณคูหา,2024-06-03T20:00:46.879Z,NaN,NaN,Nong Bua Lam Phu,Suwannakhuha,Suwannakhuha
4321,P03025734,วัดมหาชัย,"ประดิษฐาน ""พระบาง"" พระพุทธรูปสำคัญคู่บ้านคู่เม...",สถานที่ท่องเที่ยว,3,17.20738,102.438290,39000.0,NaN,NaN,...,3901.0,เมืองหนองบัวลำภู,4431.0,ลำภู,2024-06-03T20:00:33.654Z,NaN,NaN,Nong Bua Lam Phu,Mueang Nong Bua Lam Phu,Lam Phu
4322,P03010515,อนุสาวรีย์ไทยอาสาป้องกันชาติ (ทสปช.),NaN,สถานที่ท่องเที่ยว,3,17.36615,102.267420,39350.0,NaN,NaN,...,3902.0,นากลาง,4441.0,กุดดินจี่,2024-06-03T20:00:24.475Z,NaN,National Defence Volunteers Monument,Nong Bua Lam Phu,Na Klang,Kut Din Chi
4323,P03014035,วัดถ้ำสุวรรณคูหา,NaN,สถานที่ท่องเที่ยว,3,17.60756,102.282970,39270.0,NaN,NaN,...,3905.0,สุวรรณคูหา,4478.0,นาสี,2024-06-02T20:00:15.456Z,NaN,Tham Suwannakhuha Cave,Nong Bua Lam Phu,Suwannakhuha,Nasi


In [52]:
Merge_Attractions_df.to_csv('./merged_tat_attractions.csv', index=False, encoding='utf-8')